In [121]:
import sqlite3
import pandas as pd
from datetime import datetime

In [122]:
database = 'database.sqlite'
conn = sqlite3.connect(database)

In [123]:
#player_data = pd.read_sql("SELECT * FROM Player;", conn)
#leagues = pd.read_sql("Select * from League;",conn)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)


match_data = pd.read_sql("SELECT * FROM Match;", conn)
match_data.to_csv('match_data.csv',encoding='utf-8')
print(match_data.columns.values)
#print(match_data)
teams = pd.read_sql("SELECT * FROM Team;", conn)

['id' 'country_id' 'league_id' 'season' 'stage' 'date' 'match_api_id'
 'home_team_api_id' 'away_team_api_id' 'home_team_goal' 'away_team_goal'
 'home_player_X1' 'home_player_X2' 'home_player_X3' 'home_player_X4'
 'home_player_X5' 'home_player_X6' 'home_player_X7' 'home_player_X8'
 'home_player_X9' 'home_player_X10' 'home_player_X11' 'away_player_X1'
 'away_player_X2' 'away_player_X3' 'away_player_X4' 'away_player_X5'
 'away_player_X6' 'away_player_X7' 'away_player_X8' 'away_player_X9'
 'away_player_X10' 'away_player_X11' 'home_player_Y1' 'home_player_Y2'
 'home_player_Y3' 'home_player_Y4' 'home_player_Y5' 'home_player_Y6'
 'home_player_Y7' 'home_player_Y8' 'home_player_Y9' 'home_player_Y10'
 'home_player_Y11' 'away_player_Y1' 'away_player_Y2' 'away_player_Y3'
 'away_player_Y4' 'away_player_Y5' 'away_player_Y6' 'away_player_Y7'
 'away_player_Y8' 'away_player_Y9' 'away_player_Y10' 'away_player_Y11'
 'home_player_1' 'home_player_2' 'home_player_3' 'home_player_4'
 'home_player_5' 'home_pl

In [124]:
result_entries = []
feature_entries = []

for i in range(len(match_data)):
    record = match_data.loc[i, :]
    home_goals = int(record['home_team_goal'])
    away_goals = int(record['away_team_goal'])
    if home_goals > away_goals:
        result = 'HOME'
    elif away_goals > home_goals:
        result = 'AWAY'
    else:
        result = 'DRAW'
    home_team = teams[teams.team_api_id == record['home_team_api_id']]['team_long_name'].values[0]
    away_team = teams[teams.team_api_id == record['away_team_api_id']]['team_long_name'].values[0]
    home_player_X1=record['home_player_X1']
    home_player_X2=record['home_player_X2']
    home_player_X3=record['home_player_X3']
    home_player_X4=record['home_player_X4']
    home_player_X5=record['home_player_X5']
    away_player_X1=record['away_player_X1']
    away_player_X2=record['away_player_X2']
    away_player_X3=record['away_player_X3']
    away_player_X4=record['away_player_X4']
    away_player_X5=record['away_player_X5']
    home_player_Y1=record['home_player_Y1']
    home_player_Y2=record['home_player_Y2']
    home_player_Y3=record['home_player_Y3']
    home_player_Y4=record['home_player_Y4']
    home_player_Y5=record['home_player_Y5']
    away_player_Y1=record['away_player_Y1']
    away_player_Y2=record['away_player_Y2']
    away_player_Y3=record['away_player_Y3']
    away_player_Y4=record['away_player_Y4']
    away_player_Y5=record['away_player_Y5']
    

    result_entry = {'home_team_goal': record['home_team_goal'], 'away_team_goal': record['away_team_goal'],
                    'result': result, 'home_player_X1':home_player_X1,'home_player_X2':home_player_X2,'home_player_X3':home_player_X3,'home_player_X4':home_player_X4,'home_player_X5':home_player_X5,
                   'home_player_Y1':home_player_Y1,'home_player_Y2':home_player_Y2,'home_player_Y3':home_player_Y3,'home_player_Y4':home_player_Y4,'home_player_Y5':home_player_Y5 
                   }
                   
    result_entries.append(result_entry)

result_df = pd.DataFrame(result_entries)
result_df['result'].replace('HOME',int('+1'),inplace=True)
result_df['result'].replace('AWAY',int('-1'),inplace=True)
result_df['result'].replace('DRAW',int('0'),inplace=True)

print(result_df.columns.values)
#result_df.to_csv('result_v3.csv',encoding='utf-8')
        #.sort_values(by='date').tail(n_games)
    #away_all_goals_scored = sum(away_home_games['home_team_goal']) + away_away_goals_scored
    #away_all_goals_conceded = sum(away_home_games['away_team_goal']) + away_away_goals_conceded
 

['away_team_goal' 'home_player_X1' 'home_player_X2' 'home_player_X3'
 'home_player_X4' 'home_player_X5' 'home_player_Y1' 'home_player_Y2'
 'home_player_Y3' 'home_player_Y4' 'home_player_Y5' 'home_team_goal'
 'result']


In [129]:


#Simple KNN on match data
import numpy as np
from sklearn import preprocessing, cross_validation, neighbors
from sklearn.cross_validation import train_test_split
import pandas as pd 

result_df.dropna(inplace = True)
labels = result_df.loc[:,'result']
features = result_df.drop('result', axis = 1)

clf = neighbors.KNeighborsClassifier(n_neighbors=6000)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2)

print(X_train.shape)
print(y_train.shape)

clf.fit(X_train,y_train)
accuracy = clf.score(X_test,y_test)

print (accuracy)

(19317, 12)
(19317,)
0.646790890269


In [97]:
for i in range(len(match_data)):
    record = match_data.iloc[i, :]
    #print('FEATURES', str(i) + '/' + str(len(match_data) - 1))
    feature_entry = {}
    feature_entry['match_id'] = record['match_api_id']
    feature_entry['date'] = datetime.strptime(record['date'], '%Y-%m-%d %H:%M:%S')
    feature_entry['home_team'] = teams[teams.team_api_id == record['home_team_api_id']]['team_long_name'].values[0]
    feature_entry['away_team'] = teams[teams.team_api_id == record['away_team_api_id']]['team_long_name'].values[0]
    a,b,c=generate_feature_from_existing(teams[teams.team_api_id == record['home_team_api_id']]['team_long_name'].values[0],teams[teams.team_api_id == record['home_team_api_id']]['team_long_name'].values[0],datetime.strptime(record['date'], '%Y-%m-%d %H:%M:%S'),result_df.copy())
    feature_entry['home_home_goals_scored'] = a
    feature_entry['home_home_goals_conceded'] = b
    #feature_entry['home_all_goals_scored'] = c
    #feature_entry['home_all_goals_conceded'] = d
    feature_entry['home_home_points'] = c
    #feature_entry['home_all_points'] = f
    #feature_entry['away_away_goals_scored'] = d
    #feature_entry['away_away_goals_conceded'] = e
    #feature_entry['away_all_goals_scored'] = i
    #eature_entry['away_all_goals_conceded'] = j
    #feature_entry['away_away_points'] = f
    #feature_entry['away_all_points'] = l
    #feature_entry['home_mutual_points'] = m
    #feature_entry['home_mutual_goals_scored'] = n
    #feature_entry['home_mutual_goals_conceded'] = o
    #feature_entry['away_mutual_points'] = p
    #feature_entry['away_mutual_goals_scored'] = q
    #feature_entry['away_mutual_goals_conceded'] = r
    feature_entries.append(feature_entry)
    
    
feature_df = pd.DataFrame(feature_entries)

In [98]:
print(feature_df)

                     away_team       date  home_home_goals_conceded  \
0                 Beerschot AC 2008-08-17                         0   
1             Sporting Lokeren 2008-08-16                         0   
2               RSC Anderlecht 2008-08-16                         0   
3                    RAEC Mons 2008-08-17                         0   
4            Standard de Liège 2008-08-16                         0   
5               Club Brugge KV 2008-09-24                         2   
6                  KV Kortrijk 2008-08-16                         0   
7         Royal Excel Mouscron 2008-08-16                         0   
8           Sporting Charleroi 2008-08-16                         0   
9                  KV Kortrijk 2008-11-01                         2   
10          Sporting Charleroi 2008-10-31                         2   
11                    KAA Gent 2008-11-02                         3   
12                   RAEC Mons 2008-11-01                         3   
13    